In [ ]:
#проверка подключения PostgreSQL 
import psycopg2
import pandas as pd

DB_HOST = '52.157.159.24'

#####################
#!введите свои реквизиты!

DB_USER = 'student7'
DB_USER_PASSWORD = 'student7_password'
DB_NAME = 'sql_ex_for_student7'

conn = psycopg2.connect(host=DB_HOST, user=DB_USER, password=DB_USER_PASSWORD, dbname=DB_NAME)

In [4]:
#--Корабли: Для каждого класса определите число кораблей этого класса, потопленных в сражениях. 
#Вывести: класс и число потопленных кораблей.
request = """
SELECT class, count(ship) FROM ships left join Outcomes on Outcomes.ship=ships.name
where result='sunk'
group by class

"""
pd.read_sql_query(request, conn)

,class,count
0,Kongo,1


In [ ]:
#--task2
#--Корабли: Для каждого класса определите год, когда был спущен на воду первый корабль этого класса. 
#Если год спуска на воду головного корабля неизвестен, определите минимальный год спуска на воду кораблей этого класса.
#Вывести: класс, год.
request = """
select class, min(launched) from Ships GROUP BY Class
UNION
select ship as class, NULL from Outcomes 
WHERE ship IN (select class from Classes)  and ship not IN (select name from Ships)

"""
pd.read_sql_query(request, conn)



In [ ]:
#--task3
#--Корабли: Для классов, имеющих потери в виде потопленных кораблей и не менее 3 кораблей в базе данных,
#вывести имя класса и число потопленных кораблей.
request = """
select class, count(class) from
( 
select Classes.class, outcomes.ship from classes join outcomes on Classes.class=outcomes.ship where outcomes.result='sunk' 
union 
select Ships.class, Outcomes.ship from outcomes join ships on Ships.name=Outcomes.ship where Outcomes.result='sunk') as C
where class in 
	( select distinct M.class from  (select Classes.class, Outcomes.ship from classes join outcomes on Classes.class=Outcomes.ship 
	  union 
	  select Classes.class, Ships.name from classes join ships on Classes.class=Ships.class) as M group by M.class 
having count(M.class)>=3 )  group by class 

"""
pd.read_sql_query(request, conn)

In [ ]:
#--task4
#--Корабли: Найдите названия кораблей, имеющих наибольшее число орудий среди всех кораблей такого же водоизмещения 
#(учесть корабли из таблицы Outcomes).
request = """
select ships.class, count(name), max(classes.numGuns) as maximum from ships left join classes on ships.class = classes.class
left join outcomes on outcomes.ship= ships.name
group by ships.class, classes.numGuns

"""
pd.read_sql_query(request, conn)

In [ ]:
#--task5
#--Компьютерная фирма: Найдите производителей принтеров, которые производят ПК с наименьшим объемом RAM и 
#с самым быстрым процессором среди всех ПК, имеющих наименьший объем RAM. Вывести: Maker
request = """

SELECT DISTINCT maker FROM Product 
WHERE type = 'printer' 
AND maker IN 
      (select maker from Product 
where model in 
      (select model from pc 
where ram in 
      (select min(ram) from pc)
and (speed in 
      (select max(speed) from pc 
where ram in  (select min(ram) from pc))
               )))
"""
pd.read_sql_query(request, conn)